# Training program on AI/ML
## Feature selection

## Dataset loading and preprocessing

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
f=open('/content/drive/MyDrive/Datasets/Demo-notebook/sentiment.txt','r')
sentences=f.readlines()
f.close()

In [51]:
import re

stopwords = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

def text_cleaner(text):
    text=remove_link(text.lower())
    long_words=[]
    for i in text.split():
        if i not in stopwords:
            long_words.append(i)
    return long_words

def remove_link(text):
    regex = r'https?://[^\s<>)"‘’]+'
    match = re.sub(regex,' ', text)
    regex = r'https?:|urls?|[/\:,-."\'?!;…]+'
    tweet = re.sub(regex,' ', match)
    tweet = re.sub("[^a-zA-Z_]", " ", tweet)
    tweet = re.sub("[ ]+", " ", tweet)
    return tweet


In [ ]:
vocabulary=[]
X=[]
y=[]
for line in sentences:
	texts=line.strip().lower().split('\t')
	tokens=text_cleaner(texts[0])
	y.append(texts[1])
	X.append(tokens)
	for word in tokens:
		vocabulary.append(word)

## Conversion of text to vector space

In [ ]:
def t2v(tokens,vocabulary):
	vect=[]
	for feature in vocabulary:
		if feature in tokens:
			vect.append(1)
		else:
			vect.append(0)
	return vect

In [ ]:
vector=[]
for tokens in X:
	vect=t2v(tokens,vocabulary)
	vector.append(vect)

In [ ]:
import numpy as np
vector=np.asarray(vector)
y=np.asarray(y)
print("Number of samples,Number of features")
vector.shape


Number of samples,Number of features


(1143, 12213)

# Feature selection

### Mutual Information based feature selection

In [ ]:
from sklearn.feature_selection import mutual_info_classif
X_new = mutual_info_classif(vector,y)
X_new.shape

(12213,)

In [ ]:
N=len(vocabulary)
avg=0
for mi in X_new:
    avg+=mi
avg=avg/N
feat_selection=[]
for i,mi in enumerate(X_new):
    if mi>avg:
        feat_selection.append(vocabulary[i])
len(feat_selection)

4261

In [ ]:
feat_selection

['humbles',
 'exalted',
 'sought',
 'truth',
 'semst',
 'world',
 'relieve',
 'hate',
 'religion',
 'precedes',
 'semst',
 'husband',
 'wants',
 'god',
 'happy',
 'day',
 'america',
 'independenceday',
 'semst',
 'wisdom',
 'good',
 'prov',
 'cool',
 'islam',
 'just',
 'sinner',
 'strongly',
 'strongly',
 'rejoice',
 'luther',
 'god',
 'god',
 'god',
 'god',
 'semst',
 'catholic',
 'life',
 'exercises',
 'silent',
 'influence',
 'men',
 'i',
 'lord',
 'semst',
 'honour',
 'house',
 'sundaymorningshow',
 'i',
 'god',
 'god',
 'difficulties',
 'strong',
 'god',
 'll',
 'know',
 'god',
 'life',
 'ask',
 'semst',
 'jesus',
 'response',
 'royal',
 'thank',
 'god',
 'thy',
 'word',
 'truth',
 'semst',
 'psalm',
 'god',
 'sign',
 'days',
 'humanist',
 'church',
 'new',
 'kissthering',
 'stay',
 'love',
 'love',
 'know',
 'accident',
 'tcot',
 'pjnet',
 'ccot',
 'i',
 'feel',
 't',
 'away',
 'semst',
 'faith',
 'sees',
 'difficulties',
 'counts',
 'god',
 'smith',
 'bring',
 'national',
 'scho

In [ ]:
vector_feat=[]
for tokens in X:
	vect=t2v(tokens,feat_selection)
	vector_feat.append(vect)

In [ ]:
import numpy as np
vector_feat=np.asarray(vector_feat)
vector_feat.shape

(1143, 4261)

In [ ]:
test_ip="I love watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_pos=t2v(tokens,feat_selection)
tokens

['i', 'love', 'watching', 'netflix', 'original', 'movies']

In [ ]:
test_ip="I hate watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_neg=t2v(tokens,feat_selection)
tokens

['i', 'hate', 'watching', 'netflix', 'original', 'movies']

# Training Classifier

In [ ]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
print("Number of samples, Number of features")
vector_feat.shape

Number of samples,Number of features


(1143, 4261)

In [ ]:
svml=svm.LinearSVC()
model = svml.fit(vector_feat,y)

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
res = model.predict([vect_pos])
res

array(['1'], dtype='<U1')

In [ ]:
res = model.predict([vect_neg])
res

array(['0'], dtype='<U1')

In [ ]:
print("Number of samples, Number of features")
vector.shape

Number of samples, Number of features


(1143, 12213)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(vector, y)
t_model = SelectFromModel(lsvc, prefit=True)
X_new = t_model.transform(vector)
X_new.shape

(1143, 10)

In [ ]:
X_new[1]

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0])

In [ ]:
svml=svm.LinearSVC()
model = svml.fit(X_new,y)

In [ ]:
test_ip="I love watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_pos=t2v(tokens,vocabulary)
vect_pos = t_model.transform(np.asarray([vect_pos]))
vect_pos

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
res = model.predict(vect_pos)
res

array(['0'], dtype='<U1')

In [ ]:
test_ip="I hate watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_neg=t2v(tokens,vocabulary)
vect_neg = t_model.transform(np.asarray([vect_neg]))
vect_neg

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
res = model.predict(vect_neg)
res

array(['0'], dtype='<U1')

# Dimensional Reduction

### PCA based dimensional reduction

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=200)
X_new=pca.fit_transform(vector)
X_new.shape

(1143, 200)

In [ ]:
svml=svm.LinearSVC()
model = svml.fit(X_new,y)

In [ ]:
test_ip="I love watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_pos=t2v(tokens,vocabulary)
vect_pos = pca.transform(np.asarray([vect_pos]))
vect_pos.shape

(1, 200)

In [ ]:
res = model.predict(vect_pos)
res

array(['1'], dtype='<U1')

In [ ]:
test_ip="I hate watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_neg=t2v(tokens,vocabulary)
vect_neg = pca.transform(np.asarray([vect_neg]))
vect_neg.shape

(1, 200)

In [ ]:
res = model.predict(vect_neg)
res

array(['0'], dtype='<U1')

### SVD based dimensional reduction

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=200, n_iter=7, random_state=42)
X_new=svd.fit_transform(vector)
X_new.shape

(1143, 200)

In [ ]:
svml=svm.LinearSVC()
model = svml.fit(X_new,y)

In [ ]:
test_ip="I love watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_pos=t2v(tokens,vocabulary)
vect_pos = svd.transform(np.asarray([vect_pos]))
vect_pos.shape

(1, 200)

In [ ]:
test_ip="I hate watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_neg=t2v(tokens,vocabulary)
vect_neg = svd.transform(np.asarray([vect_neg]))
vect_neg.shape

(1, 200)

In [ ]:
res = model.predict(vect_pos)
res

array(['1'], dtype='<U1')

In [ ]:
res = model.predict(vect_neg)
res

array(['1'], dtype='<U1')

# Word2Vec

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 40.3 MB/s eta 0:00:00


In [ ]:
import gensim, logging, os
from gensim import corpora
from collections import defaultdict
from pprint import pprint
import numpy as np
from gensim.models import Word2Vec
import time
t1 = time.time()

In [ ]:
sentences[0]

'He who exalts himself shall      be humbled; and he who humbles himself shall be exalted.Matt 23:12.     #SemST\t1\n'

In [ ]:
texts = [[word for word in text_cleaner(line.strip().split('\t')[0])] for line in sentences]

In [ ]:
texts

[['exalts',
  'shall',
  'humbled',
  'humbles',
  'shall',
  'exalted',
  'matt',
  'semst'],
 ['brainman',
  'heidtjj',
  'benjaminlives',
  'i',
  'sought',
  'truth',
  'soul',
  'strong',
  'stand',
  'merits',
  'semst'],
 ['world',
  'needs',
  'tight',
  'group',
  'hug',
  'tight',
  'relieve',
  'anger',
  'hate',
  'makepeacewitheachother',
  'semst'],
 ['morality',
  'derived',
  'religion',
  'precedes',
  'christopher',
  'hitch',
  'hitchens',
  'freethinkers',
  'semst'],
 ['godly',
  'husband',
  'knows',
  'trusts',
  'loves',
  'respects',
  'honors',
  'supports',
  'wants',
  'appreciates',
  'god',
  'semst'],
 ['happy',
  'independence',
  'day',
  'america',
  'beautiful',
  'constitution',
  'independenceday',
  'usa',
  'semst'],
 ['let',
  'house',
  'built',
  'wisdom',
  'strong',
  'good',
  'sense',
  'prov',
  'semst'],
 ['days', 'cool', 'kids', 'atheists', 'freethinker', 'semst'],
 ['iloveislambecause',
  'v',
  'love',
  'islam',
  'deen',
  'instead',

In [ ]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
model = Word2Vec(
    sentences=texts,
    min_count=0,
    sample=0.001,
    seed=1,
    workers=8,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    negative=5,
    epochs=100,
    vector_size=50
)

In [ ]:
vector = model.wv['unjustified']
print(vector)

[-0.01491481 -0.01342598  0.01197997  0.12943293 -0.01135988 -0.1624539
  0.2638394   0.33107078 -0.43656528  0.01702213 -0.04289902 -0.04846508
  0.08588639  0.04631686 -0.02868883  0.11512707  0.09342771  0.10074719
 -0.33349195 -0.23278467  0.11463168  0.35671264  0.46809316 -0.1839103
  0.25362095  0.05101835 -0.11831456 -0.07039364 -0.1400729   0.07703312
  0.0230593  -0.01898804  0.037189   -0.10564873  0.04362874  0.08936709
  0.10701859  0.08131867 -0.04086617 -0.02333345  0.09892934  0.21633476
 -0.13677742  0.0553967   0.5335584   0.09273511 -0.16985856 -0.18072875
  0.10488815  0.07388455]


In [ ]:
sims = model.wv.most_similar('responsible', topn=10)
for word, score in sims:
    print(f"{word}: {score:.4f}")

course: 0.8927
wether: 0.8790
chooseliferacer: 0.8739
continuum: 0.8683
sentient: 0.8640
toby_dorena: 0.8625
drunk: 0.8616
autonomous: 0.8609
teen: 0.8590
uterus: 0.8587


In [72]:
result = model.wv.most_similar(positive=['healthcare', 'clinics'], negative=['death'], topn=1)
print(result) # Expected output: [('queen', 0.7118)]

[('surprised', 0.8755826354026794)]


# FastText

In [52]:
from gensim.models.fasttext import FastText

In [58]:
!pip install nltk
!pip install emoji

In [55]:
stopwords = ['rt','amp','url','sir','day','title','shri','crore','time',"a", "about","above", "across", "after", "afterwards", "again", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]
tstop = ['rt','amp','url']
import itertools

import re

def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r"'s\b"," ",newString)
    newString = re.sub("[^a-zA-Z_@#]", " ", newString)
    long_words=[]
    for i in newString.split():
        if i not in tstop:
            long_words.append(i)
    return long_words

import emoji

def remove_link(text):
    regex = r'https?://[^\s<>)"‘’]+'
    match = re.sub(regex,' ', text)
    regex = r'urls?'
    match = re.sub(regex,' ', match)
    tweet=emoji.demojize(match)
    tweet = re.sub("[0-9:;,.()!?…]", " ", tweet)
    tweet = re.sub("[ ]+", " ", tweet)
    return tweet.strip()

In [57]:
import nltk

In [59]:
word_punctuation_tokenizer = nltk.WordPunctTokenizer()
word_tokenized_corpus = [word_punctuation_tokenizer.tokenize(sent) for sent in sentences]

In [62]:
embedding_size = 60
window_size = 10
min_word = 5
down_sampling = 1e-2

In [61]:
ft_model = FastText(
    sentences=word_tokenized_corpus,
    vector_size=embedding_size,
    window=window_size,
    min_count=min_word,
    sample=down_sampling,
    sg=1,
    epochs=100
)

In [65]:
ft_model.wv.most_similar(['responsible'], topn=10)

[('abortions', 0.5839306712150574),
 ('while', 0.5687960386276245),
 ('decisions', 0.5594754219055176),
 ('re', 0.5495399832725525),
 ('big', 0.5388058423995972),
 ('especially', 0.5200111269950867),
 ('says', 0.5176717042922974),
 ('questions', 0.516128420829773),
 ('and', 0.5112561583518982),
 ('Right', 0.5070515275001526)]

In [73]:
result = ft_model.wv.most_similar(positive=['healthcare', 'clinics'], negative=['death'], topn=1)

print(f"Analogy result: {result[0][0]}")

Analogy result: Bernie2016
